# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
from numpy.random import seed
seed(3)
from tensorflow import set_random_seed
set_random_seed(3)

#You can find the answer at Keras docs: https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development.

#In short, to be absolutely sure that you will get reproducible results with your python script on one computer's/laptop's CPU then you will have to do the following:

#Set PYTHONHASHSEED environment variable at a fixed value
#Set python built-in pseudo-random generator at a fixed value
#Set numpy pseudo-random generator at a fixed value
#Set tensorflow pseudo-random generator at a fixed value
#Configure a new global tensorflow session
#Following the Keras link at the top, the source code I am using is the following:

# Seed value
# Apparently you may use different seed values at each stage
seed_value= 3

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [67]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [21]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9884206550568358204
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
english_sentences = helper.load_data('data/small_vocab_en')
french_sentences = helper.load_data('data/small_vocab_fr')
print('Dataset Loaded')

Dataset Loaded


In [11]:
type(english_sentences2)

list

In [12]:
len(english_sentences2)

137861

In [22]:
import os
 
dirpath = os.getcwd()
print("current directory is : " + dirpath)
foldername = os.path.basename(dirpath)
print("Directory name is : " + foldername)

current directory is : /root/mount/NMT
Directory name is : NMT


In [3]:
import pandas as pd
df = pd.read_csv ('data/DescAB.csv')   #read the csv file (put 'r' before the path string to address any special characters in the path, such as '\'). Don't forget to put the file name at the end of the path + ".csv"
df.info()

/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49722 entries, 0 to 49721
Data columns (total 26 columns):
sku                      49722 non-null object
marca                    49722 non-null object
id_marca                 49722 non-null int64
modelo                   49674 non-null object
id_modelo                49722 non-null int64
descricao_concorrente    49722 non-null object
id_produto_detalhe       49722 non-null int64
custo                    41732 non-null float64
data_cadastro            49722 non-null object
descricao                49722 non-null object
ean                      49722 non-null int64
frete                    41732 non-null object
lista                    49722 non-null object
cod_ref                  49722 non-null object
id_usuario               0 non-null float64
loja                     49722 non-null object
id_loja                  49722 non-null int64
segmento                 49722 non-null object
id_segmento              49722 non-null int64
sub_se

In [94]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


In [4]:
df=df.sample(frac=1) #Shuffle the dataframe

In [5]:
df.shape

(49722, 26)

In [6]:
df=df.sample(frac=0.2)

In [7]:
df = df.reset_index(drop=True)

In [8]:
df.shape

(9944, 26)

In [9]:
df[["descricao","descricao_concorrente"]].head(3)

,descricao,descricao_concorrente
0,TORRADEIRA DG 22702-BZ220 INOX 220V HAMILTON B...,TORRADEIRA DIGITAL HAMILTON BEACH
1,REF FF 402L ELECTROLUX DF44S 220V PLATINUM,GELADEIRA ELECTROLUX FROST FREE DUPLEX PLATINU...
2,LAVADORA DE ROUPAS PRAXIS TWIN TUB LAVA E CENT...,"TANQUINHO PRAXIS TWIN TUB 4,1KG - 2 EM 1 LAVA ..."


In [10]:
english_sentences = df[["descricao"]]
english_sentences = english_sentences.descricao
french_sentences=df[["descricao_concorrente"]]
french_sentences=french_sentences.descricao_concorrente

### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [102]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  EDREDOM SOFT COMFORT QUEEN BRANCO
small_vocab_fr Line 1:  EDREDOM QUEEN PLUMASUL SOFT COMFORT 240X260CM MICROFIBRA BRANCO
small_vocab_en Line 2:  CAMA DE CASAL S819 BR KAPPESBERG
small_vocab_fr Line 2:  CAMA DE CASAL S819 KAPPESBERG BRANCO


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [103]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

295111 English words.
14054 unique English words.
10 Most common words in the English dataset:
"DE" "-" "220V" "BR" "E" "COM" "PRETO" "TRAMONTINA" "127V" "INOX"

390483 French words.
15213 unique French words.
10 Most common words in the French dataset:
"-" "DE" "COM" "E" "BRANCO" "PARA" "PRETO" "INOX" "TRAMONTINA" "2"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [104]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [105]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [106]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 25
Max French sentence length: 32
English vocabulary size: 13545
French vocabulary size: 13669


In [107]:
preprocess_x, x_tk = tokenize(english_sentences)
preprocess_y, y_tk = tokenize(french_sentences)
print(y_tk)
print(x_tk)
print(x_tk.word_index)

{'de': 1, '220v': 2, 'br': 3, 'preto': 4, 'e': 5, 'com': 6, 'inox': 7, 'tramontina': 8, '127v': 9, '110v': 10, 'branco': 11, 'c': 12, 'pr': 13, 'electrolux': 14, '2': 15, '1': 16, 'philco': 17, 'para': 18, 'cel': 19, 'desb': 20, '3': 21, 'p': 22, '6': 23, '4': 24, 'bc': 25, 'tv': 26, 'azul': 27, 'vm': 28, 'brastemp': 29, 'samsung': 30, 'brinox': 31, 'ref': 32, 'pt': 33, 'fios': 34, 'ff': 35, 'biv': 36, 'mondial': 37, 'black': 38, '5': 39, 'pecas': 40, 'smart': 41, 'a': 42, 'led': 43, 'jogo': 44, 'cm': 45, 'em': 46, 'oster': 47, 'lg': 48, 'mesa': 49, 'premium': 50, 'porta': 51, 'kit': 52, 'consul': 53, '4k': 54, 'galaxy': 55, 's': 56, 'x': 57, 'l': 58, 'caixa': 59, 'plus': 60, 'cama': 61, 'plumasul': 62, 'preta': 63, 'britania': 64, 'usb': 65, 'col': 66, 'mini': 67, '0': 68, 'pcs': 69, 'pto': 70, 'vermelho': 71, '8': 72, '32gb': 73, 'uhd': 74, 'panela': 75, 'pro': 76, 'forno': 77, 'prata': 78, 'som': 79, 'casal': 80, 'cadeira': 81, 'az': 82, 'touch': 83, '12': 84, 'fio': 85, 'not': 86, 

In [108]:
print(y_tk.word_index)

{'de': 1, 'com': 2, 'e': 3, 'branco': 4, 'preto': 5, '2': 6, 'para': 7, 'inox': 8, 'tramontina': 9, '4': 10, '3': 11, '1': 12, '6': 13, '5': 14, 'pecas': 15, 'electrolux': 16, 'usb': 17, 'a': 18, 'philco': 19, 'plumasul': 20, 'smartphone': 21, 'tela': 22, 'litros': 23, 'jogo': 24, 'brinox': 25, 'samsung': 26, '8221': 27, 'eletrica': 28, 'cm': 29, 'tv': 30, 'bocas': 31, 'smart': 32, '4g': 33, 'azul': 34, '127v': 35, 'cam': 36, 'brastemp': 37, '220v': 38, 'frost': 39, 'free': 40, 'geladeira': 41, 'bluetooth': 42, 'casal': 43, 'branca': 44, 'mondial': 45, 'fios': 46, 'conjunto': 47, 'velocidades': 48, 'colchao': 49, '8': 50, 'portas': 51, 'mesa': 52, '10': 53, 'em': 54, 'fi': 55, 'kit': 56, 'x': 57, 'wi': 58, '110v': 59, 'porta': 60, 'selfie': 61, 'solteiro': 62, 'digital': 63, 'refrigerador': 64, 'black': 65, 'vermelho': 66, 'touch': 67, 'camera': 68, 'moveis': 69, 'led': 70, '12': 71, 'premium': 72, 'aco': 73, 'britania': 74, 'cama': 75, 'lg': 76, 'dual': 77, 'portatil': 78, 'hdmi': 79,

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [109]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [110]:
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

In [111]:
tmp_x[1]

array([[  61],
       [   1],
       [  80],
       [5522],
       [   3],
       [  98],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0],
       [   0]], dtype=int32)

In [112]:
tmp_x.shape

(39778, 32, 1)

In [113]:
preproc_french_sentences.shape

(39778, 32, 1)

In [114]:
tmp_x.shape

(39778, 32, 1)

### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [46]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [47]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size+1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 3977 samples, validate on 995 samples
Epoch 1/10
3977/3977 [==============================] - 226s 57ms/step - loss: 8.8225 - acc: 0.2666 - val_loss: 8.7824 - val_acc: 0.4730
Epoch 2/10
3977/3977 [==============================] - 229s 57ms/step - loss: 8.7546 - acc: 0.5533 - val_loss: 8.7152 - val_acc: 0.6375
Epoch 3/10
3977/3977 [==============================] - 231s 58ms/step - loss: 8.6801 - acc: 0.6383 - val_loss: 8.6368 - val_acc: 0.6430
Epoch 4/10
3977/3977 [==============================] - 232s 58ms/step - loss: 8.5899 - acc: 0.6425 - val_loss: 8.5322 - val_acc: 0.6450
Epoch 5/10
3977/3977 [==============================] - 232s 58ms/step - loss: 8.4620 - acc: 0.6434 - val_loss: 8.3646 - val_acc: 0.6449
Epoch 6/10
3977/3977 [==============================] - 241s 61ms/step - loss: 8.2435 - acc: 0.6434 - val_loss: 8.0401 - val_acc: 0.6451
Epoch 7/10
3977/3977 [==============================] - 239s 60ms/step - loss: 7.7966 - acc: 0.6439 - val_loss: 7.3468 - val_acc: 0

In [32]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 87s 785us/step - loss: 3.4516 - acc: 0.4162 - val_loss: 2.5509 - val_acc: 0.4610
Epoch 2/10
110288/110288 [==============================] - 93s 845us/step - loss: 2.4338 - acc: 0.4711 - val_loss: 2.3132 - val_acc: 0.4867
Epoch 3/10
110288/110288 [==============================] - 97s 881us/step - loss: 2.1915 - acc: 0.5061 - val_loss: 2.0614 - val_acc: 0.5294
Epoch 4/10
110288/110288 [==============================] - 93s 844us/step - loss: 1.9480 - acc: 0.5480 - val_loss: 1.8417 - val_acc: 0.5679
Epoch 5/10
110288/110288 [==============================] - 94s 853us/step - loss: 1.7689 - acc: 0.5768 - val_loss: 1.7062 - val_acc: 0.5850
Epoch 6/10
110288/110288 [==============================] - 92s 836us/step - loss: 1.6624 - acc: 0.5858 - val_loss: 1.6234 - val_acc: 0.5852
Epoch 7/10
110288/110288 [==============================] - 94s 849us/step - loss: 1.5933 - acc: 0.5902

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [79]:
from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network

embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)

embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=11, validation_split=0.2)


# TODO: Print prediction(s)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

/opt/conda/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


Train on 3977 samples, validate on 995 samples
Epoch 1/11
3977/3977 [==============================] - 74s 19ms/step - loss: 8.8333 - acc: 0.3198 - val_loss: 8.8207 - val_acc: 0.6569
Epoch 2/11
3977/3977 [==============================] - 72s 18ms/step - loss: 8.8116 - acc: 0.6607 - val_loss: 8.7930 - val_acc: 0.6645
Epoch 3/11
3977/3977 [==============================] - 72s 18ms/step - loss: 8.7780 - acc: 0.6643 - val_loss: 8.7458 - val_acc: 0.6621
Epoch 4/11
3977/3977 [==============================] - 75s 19ms/step - loss: 8.7185 - acc: 0.6610 - val_loss: 8.6582 - val_acc: 0.6601
Epoch 5/11
3977/3977 [==============================] - 71s 18ms/step - loss: 8.6046 - acc: 0.6600 - val_loss: 8.4830 - val_acc: 0.6599
Epoch 6/11
3977/3977 [==============================] - 71s 18ms/step - loss: 8.3672 - acc: 0.6597 - val_loss: 8.0951 - val_acc: 0.6599
Epoch 7/11
3977/3977 [==============================] - 71s 18ms/step - loss: 7.8368 - acc: 0.6597 - val_loss: 7.2954 - val_acc: 0.6599
E

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [100]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)


bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

/opt/conda/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


Train on 11933 samples, validate on 2984 samples
Epoch 1/20
11933/11933 [==============================] - 466s 39ms/step - loss: 9.0107 - acc: 0.6336 - val_loss: 8.5246 - val_acc: 0.6892
Epoch 2/20
11933/11933 [==============================] - 449s 38ms/step - loss: 7.1348 - acc: 0.6910 - val_loss: 4.8177 - val_acc: 0.7007
Epoch 3/20
11933/11933 [==============================] - 445s 37ms/step - loss: 3.5168 - acc: 0.7024 - val_loss: 2.9073 - val_acc: 0.7023
Epoch 4/20
11933/11933 [==============================] - 443s 37ms/step - loss: 2.7808 - acc: 0.7022 - val_loss: 2.6309 - val_acc: 0.7024
Epoch 5/20
11933/11933 [==============================] - 444s 37ms/step - loss: 2.5385 - acc: 0.7034 - val_loss: 2.4993 - val_acc: 0.7041
Epoch 6/20
11933/11933 [==============================] - 445s 37ms/step - loss: 2.4532 - acc: 0.7043 - val_loss: 2.4426 - val_acc: 0.7047
Epoch 7/20
11933/11933 [==============================] - 447s 37ms/step - loss: 2.4040 - acc: 0.7044 - val_loss: 2.4

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [109]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=3, validation_split=0.2)

print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

/opt/conda/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


Train on 11933 samples, validate on 2984 samples
Epoch 1/3
11933/11933 [==============================] - 441s 37ms/step - loss: 9.1830 - acc: 0.6421 - val_loss: 8.6440 - val_acc: 0.7023
Epoch 2/3
11933/11933 [==============================] - 421s 35ms/step - loss: 6.4657 - acc: 0.7020 - val_loss: 4.7795 - val_acc: 0.7023
Epoch 3/3
11933/11933 [==============================] - 424s 36ms/step - loss: 3.8669 - acc: 0.7020 - val_loss: 3.0167 - val_acc: 0.7023
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [115]:
english_vocab_size

13545

In [116]:
tmp_x.shape[1]

32

In [117]:
len(preproc_french_sentences)

39778

In [118]:
len(english_tokenizer.word_index)+1

13546

In [119]:
len(french_tokenizer.word_index)+1

13670

In [120]:
from keras.models import Sequential
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')


Final Model Loaded


/opt/conda/lib/python3.6/site-packages/keras/engine/sequential.py:109: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


## Prediction (IMPLEMENTATION)

In [121]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    tmp_X = pad(preproc_english_sentences)
    model = model_final(tmp_X.shape,
                        preproc_french_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(french_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_french_sentences, batch_size = 1024, epochs = 3, validation_split = 0.1)
    return model
 

In [122]:
model=final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 35800 samples, validate on 3978 samples
Epoch 1/3
35800/35800 [==============================] - 1065s 30ms/step - loss: 3.4540 - acc: 0.6832 - val_loss: 2.8973 - val_acc: 0.7023
Epoch 2/3
35800/35800 [==============================] - 1024s 29ms/step - loss: 2.9008 - acc: 0.7031 - val_loss: 2.9053 - val_acc: 0.7023
Epoch 3/3
35800/35800 [==============================] - 1026s 29ms/step - loss: 2.9063 - acc: 0.7031 - val_loss: 2.9214 - val_acc: 0.7023


In [63]:
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'cerv exp vert c visor 324 litros vn28fe 110v'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
    sentences = np.array([sentence[0], preproc_english_sentences[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in preproc_french_sentences[0]]))

Sample 1:
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
<PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
cervejeira e expositor vertical c visor metalfrio preto adesivado 296l vn28fe 220v <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [65]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 25, 128)           1337984   
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 512)               591360    
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 32, 512)           0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 32, 512)           1181184   
_________________________________________________________________
time_distributed_11 (TimeDis (None, 32, 10622)         5449086   
Total params: 8,559,614
Trainable params: 8,559,614
Non-trainable params: 0
_________________________________________________________________


## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [18]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 345566 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?

In [11]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 26469)
df2=pd.concat([df.descricao,df.descricao_concorrente], axis=0)
tokenizer.fit_on_texts(df2)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(df2)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 12302 unique tokens.


In [129]:
sequences[0:5]

[[179, 137, 587, 131, 4],
 [59, 1, 47, 6238, 21, 118],
 [5345, 5346, 2054],
 [1097, 1772, 394, 1174, 7690, 1145, 32, 754, 290],
 [294, 418, 209, 63, 5347, 92, 127]]

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
df2[0:10]

0    TORRADEIRA DG 22702-BZ220 INOX 220V HAMILTON B...
1           REF FF 402L ELECTROLUX DF44S 220V PLATINUM
2    LAVADORA DE ROUPAS PRAXIS TWIN TUB LAVA E CENT...
3            LAVADORA CONSUL CWE13ABA BRANCA 13KG 127V
4                          MAMADEIRA CLASSIC 330ML TRP
5                     CELULAR DESB Q6 32GB PLATINUM LG
6        PNEU 2062 TIMBERLINE PR 255 75 R15 TIMBERLINE
7     CAIXA PLASTICA INJETADA 20 AM E PRETA TRAMONTINA
8              CONJ  PANELAS PARIS 5 PCS VM TRAMONTINA
9                 MULTIFUNCIONAL CANON MEGA TANK G3110
dtype: object

In [13]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df2)
print(vectorizer.get_feature_names())

['00', '000', '00009', '0001', '00010', '0004', '000445', '0007', '0008', '000v', '000w', '001', '002', '003', '003276', '003320', '003342', '003522', '003524', '003538', '0037', '00370100', '004', '0040', '0043', '00432', '0045', '004722', '004731r', '0050', '007', '0073', '0076', '0079', '008', '009', '0095', '00m', '00x1', '00x2', '00x3', '01', '010', '0106', '011', '0113', '0117', '012', '01250', '0129', '013', '01379', '014', '01402', '01428', '015', '01608', '01658', '01688', '01698', '017', '01706', '0175', '018', '0181', '0182', '0183', '0184', '019', '01979', '02', '020', '0203', '021', '0210', '02162', '022', '023', '024', '02442', '02443', '025', '026', '02623', '02708', '02716', '028', '0280', '03', '030', '032', '03210150', '03249', '032cm', '033011081', '035', '03516', '03606', '03607', '03616', '0373', '03746', '0381', '0389', '03900', '04', '040', '0410025', '043', '0436514xtfbmper', '044015zxflg', '0441614xpe', '0441714xpe', '0447214xtper', '0457', '0460', '0461', '047

In [44]:
X

<19888x12214 sparse matrix of type '<class 'numpy.int64'>'
	with 155496 stored elements in Compressed Sparse Row format>

In [14]:
print(X.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [15]:
len(X.toarray()[1])

12214

In [16]:
len(df.descricao)


9944

In [26]:
original=X.toarray()[0:50]

In [27]:
original.shape

(50, 12214)

In [28]:
len(original)

50

In [29]:
Trad=X.toarray()[9944:9994]

In [43]:
import numpy as np
np.dot(original[1],Trad[49])

2

In [33]:
np.dot(original,Trad.transpose())

array([[3, 0, 0, ..., 0, 0, 0],
       [0, 3, 0, ..., 0, 0, 2],
       [0, 0, 6, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 3, 0, 0],
       [0, 0, 0, ..., 0, 2, 0],
       [0, 1, 0, ..., 0, 0, 3]], dtype=int64)

In [34]:
%whos

Variable            Type               Data/Info
------------------------------------------------
CountVectorizer     type               <class 'sklearn.feature_e<...>on.text.CountVectorizer'>
Tokenizer           type               <class 'keras_preprocessing.text.Tokenizer'>
Trad                ndarray            50x12214: 610700 elems, type `int64`, 4885600 bytes (4.659271240234375 Mb)
X                   csr_matrix           (0, 4056)	1\n  (0, 7182<...>589)	1\n  (19887, 5621)	1
df                  DataFrame                       sku         <...>n[9944 rows x 26 columns]
df2                 Series             0       TORRADEIRA DG 227<...>gth: 19888, dtype: object
english_sentences   Series             0       TORRADEIRA DG 227<...>ngth: 9944, dtype: object
french_sentences    Series             0                       T<...>ngth: 9944, dtype: object
helper              module             <module 'helper' from '/r<...>oot/mount/NMT/helper.py'>
np                  module             

In [ ]:
def matrixCalculation(i: int,
                      row: pd.Series,
                      data_train: pd.DataFrame,
                      smatrix_train: np.array,
                      smatrix_test: np.array) -> Tuple[np.array, np.array]:
    """

    The function calculate the probability of each description and also return
    a mask to strainer the original database

    :param i: the index product
    :param row: X values of the test data
    :param data_train: pd.DataFrame with the traning examples
    :param smatrix_train: train sparse matrix of sparseMatrix function
    :param smatrix_test: test sparse matrix of sparseMatrix function
    :return: the probability, mask and strainer

    """

    strainer = ((data_train['brand'] == row['brand']) &
               ((data_train['qunt'] == row['qunt']) |
               (data_train['unit'] == row['unit']) |
               (data_train['department'] == row['department']) |
               (data_train['category'] == row['category'])))

    if np.sum(strainer):
        pass
    else:
        strainer = ((data_train['brand'] == row['brand']) |
                   (data_train['qunt'] == row['qunt']) |
                   (data_train['unit'] == row['unit']) |
                   (data_train['department'] == row['department']) |
                   (data_train['category'] == row['category']))

    # Select the product in the sparse matrix
    srow = smatrix_test[i]

    # Multiply the sparse matrix by the select product, the words matched
    # have value of one, sum the matrix by rows
    results = np.sum(srow * smatrix_train[strainer], axis=1)

    # The highest value matched
    top = np.unique(np.sort(results))[-1]

    # The probability to each product in the train database
    prob = (results / np.sum(srow)) * 100

    # Mask to strainer the original database
    mask = np.ma.masked_greater_equal(results, top).mask

    return prob, mask, strainer


In [ ]:
def prediction(i: int,
               database_dict: Dict[str, str],
               data_train: pd.DataFrame,
               data_test: pd.DataFrame,
               strainer: pd.Series,
               mask: np.array,
               prob: np.array) -> Tuple[str, Any]:
    """

    The function predict wich itens in the train database correspond to the
    product

    :param i: the index product
    :param database_dict: dic to insert the information
    :param data_train: train database from sparseMatrix function
    :param data_test: test database from sparseMatrix function
    :param strainer: array with the first strainer
    :param mask: array mask of product
    :param prob: array probability of product
    :return: database id of the product and product

    """

    # Get databse_id of the product
    database_id = database_dict[data_test.loc[i, "seller"]]

    # Add column prob in the dataframe
    data_train.loc[strainer, "prob"] = prob

    # Convert the product to a type dict
    item = data_test.loc[i].to_dict()

    # Remove keys
    [item.pop(key) for key in ["seller", "qunt", "unit"]]

    # strainer the dataframe using mask
    info = data_train.loc[strainer, ['ean', 'sku', 'description', 'brand',
                                     'url', 'image', 'seller',
                                     'prob']].loc[mask]

    # Sort the itens according probability and description
    info = info.sort_values(by=["prob", "description"], ascending=False)

    # FIll NaN with blanck
    info = info.fillna(" ")

    # Convert the itens strainer to a dict
    info = info.to_dict("index")

    # add the itens strainer in the dict
    item["info"] = json.dumps(info)

    return database_id, item


def download_file(file_names: List[str]) -> None:
    """Upload a file to an S3 bucket

    :param files_name: file names to download from S3
    :return: None
    """

    # Download the file
    s3_client = boto3.client('s3')

    for file in file_names.values():

        s3_client.download_file("eiprice.datascience",
                                "ean_classifier/" + file,
                                file)
        print(f"File {file} was downloaded")


